In [ ]:
"""
Created on Wed Apr 13 14:17 2022

Prepare vertical profiles of T and S - not working like I want, need to think about it a bit

Author: @claraburgard
"""

In [ ]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm
import basal_melt_neural_networks.data_formatting as dfmt

from dask import delayed

import distributed
import glob

READ IN DATA

In [ ]:
nemo_run = 'OPM021'

In [ ]:
inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
outputpath_simple = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
inputpath_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
outputpath = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/'
plot_path = '/bettik/burgardc/PLOTS/first-look/'

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

chunk_size = 300

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

In [ ]:
#file_TS_orig = xr.open_mfdataset(inputpath_profiles+'T_S_profiles_per_iceshelf_1980-2018.nc', chunks={'Nisf': 1})
#file_TS_orig = xr.open_mfdataset(inputpath_profiles+'T_S_mean_prof_km_1980-2018.nc', chunks={'Nisf': 1})
#file_TS_orig = xr.open_mfdataset(inputpath_profiles+'T_S_mean_prof_km_contshelf_1980-2018.nc', chunks={'Nisf': 1})
file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018_oneFRIS.nc')
file_TS = file_TS_orig.sel(Nisf=large_isf)
file_TS_dom = file_TS.sel(profile_domain=50).drop('profile_domain')

In [ ]:
#file_TS_cut_bot = file_TS_dom.where(file_TS_dom.depth < file_isf['front_bot_depth_max'].max(), drop=True) # to keep in case I go for offshore profiles at some point
file_TS_cut_bot = file_TS_dom.dropna('depth', how='all')

In [ ]:
file_TS_cut_top_bot = xr.concat([file_TS_cut_bot.isel(depth=0),file_TS_cut_bot.where(file_TS_dom.depth >= 100, drop=True)], dim='depth')

In [ ]:
Tmin = file_TS_cut_top_bot['theta_ocean'].min('depth')
Tmax = file_TS_cut_top_bot['theta_ocean'].max('depth')
upper_depth = file_TS_cut_top_bot.depth.where(file_TS_cut_top_bot['theta_ocean'] == Tmin).min('depth')
lower_depth = file_TS_cut_top_bot.depth.where(file_TS_cut_top_bot['theta_ocean'] == Tmax).min('depth')
Smin = file_TS_cut_top_bot['salinity_ocean'].sel(depth=upper_depth)
Smax = file_TS_cut_top_bot['salinity_ocean'].sel(depth=lower_depth)

In [ ]:
Tmin

In [ ]:
upper_depth = file_TS_cut_top_bot.depth.where(file_TS_cut_top_bot['theta_ocean'] == Tmin).min('depth')
lower_depth = file_TS_cut_top_bot.depth.where(file_TS_cut_top_bot['theta_ocean'] == Tmax).min('depth')

In [ ]:
plt.figure()
upper_depth.sel(Nisf=10).plot()
lower_depth.sel(Nisf=10).plot()

In [ ]:
depth_dim = np.arange(file_TS_cut_top_bot.depth.isel(depth=np.arange(1,len(file_TS_cut_top_bot.depth))).min(),file_TS_cut_top_bot.depth.max(), 10)
interp_TS = file_TS_cut_top_bot.interp(depth=depth_dim).ffill('depth')

In [ ]:
# AUTOCORRELATION IN TEMPERATURE
corr_list = []
lag_range = np.arange(0,200,10)
for lag in lag_range:
    auto_corr = xr.corr(interp_TS['theta_ocean'],interp_TS['theta_ocean'].shift(depth=lag),dim='depth')
    corr_list.append(auto_corr)
corr_all = xr.concat(corr_list, dim='lag')

In [ ]:
plotted_var = corr_all.isel(time=0)

x = np.arange(len(lag_range))
y = np.arange(len(plotted_var.Nisf))

fig, axs = plt.subplots(1, 1,figsize=(len(y)/4,len(x)/2.75))
ax0 = axs.imshow(plotted_var.values, cmap=plt.cm.coolwarm, vmin=-0.5, vmax=1)

for j, dom in enumerate([50]):
    axs.yaxis.tick_right()
    axs.set_yticks(x)
    axs.yaxis.tick_right()

    axs.xaxis.tick_top()    
    axs.set_xticks(y)


cb_ax = fig.add_axes([0.15, 0.08, 0.7, 0.02])
#cb_ax = fig.add_axes([0.01, 0.2, 0.02, 0.7])
cbar = fig.colorbar(ax0, cax=cb_ax, orientation='horizontal')
plt.tight_layout()
